<a href="https://colab.research.google.com/github/dnlpy/d/blob/main/aiio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # [🔻] Pertama
#versions = "v1.6.0" #@param ["v1.6.0","v1.5.2","v1.5.1","v1.5.0"]

sd_model = "none"  #@param ["none", "RealisticVisionV5.safetensors", "majicMIXrealistic.safetensors", "Dreamshaper.safetensors", "Delibrate.safetensors", "RevAnimated.safetensors", "MeinaMix.safetensors", "MeinaMix.safetensors", "Delibrate.safetensors", "DarkshushiMix.safetensors", "chilloutmix_NiPrunedFp16Fix.safetensors", "AnythingV5V3_v5PrtRE.1.safetensors", "Anything_inkV5.safetensors", "ProtoGen_X5.safetensors"]
sd_vae = "vae-ft-mse-840000-ema-pruned.ckpt" #@param ["None", "WD-v2.vae.pt", "blessed-fix.vae.pt", "blessed.vae.pt", "blessed2.vae.pt", "clearvae_main.safetensors", "cute_vae.safetensors", "grapefruitVAE_v1.pt", "kl-f8-anime.ckpt", "kl-f8-anime2.ckpt", "nai.vae.pt", "orangemix.vae.pt", "pastel-waifu-diffusion.vae.pt", "rmada-cold-vae.ckpt", "vae-ft-mse-840000-ema-pruned.ckpt", "vaeextremecolors_v10.pt"]
upscalers = "4x-UltraSharp.pth" #@param ["None", "4x-UltraSharp.pth", "4x_CountryRoads_377000_G.pth", "4x_Fatality_Comix_260000_G.pth", "4x_RealisticRescaler_100000_G.pth", "4x_Valar_v1.pth", "4x_fatal_Anime_500000_G.pth", "4x_foolhardy_Remacri.pth", "A_ESRGAN_Single.pth", "LADDIER1_282500_G.pth", "WaifuGAN_v3_30000.pth", "lollypop.pth"]

import time
import concurrent.futures
import subprocess
import os
import shutil
from datetime import timedelta
from google.colab import drive
from IPython.utils import capture
from subprocess import getoutput
from urllib.parse import unquote
from google.colab.output import eval_js
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!apt-get -y install -qq aria2
#!git clone -b {versions} --single-branch https://github.com/joebayiden/stable-wibu A666
!git clone https://github.com/joebayiden/stable-wibu A666

cp_path = "/content/drive/MyDrive/A666/checkpoint"
lora_path = "/content/drive/MyDrive/A666/Lora"
vae_path = "/content/drive/MyDrive/A666/VAE"
embed_path = "/content/drive/MyDrive/A666/embeddings"
ex_path = "/content/drive/MyDrive/A666/extensions"

models_path = "/content/A666/models/Stable-diffusion"
vaes_path = "/content/A666/models/VAE"
loras_path = "/content/A666/models/Lora"
embeds_path = "/content/A666/embeddings"
extensions_path = "/content/A666/extensions"
ad_path = "/content/A666/models/adetailer"
os.makedirs (ad_path, exist_ok=True)

#URLs Links
sd_url = {
    "RealisticVisionV5.1.safetensors"         : "https://civitai.com/api/download/models/130072",
    "majicMIXrealistic.safetensors"     : "https://civitai.com/api/download/models/126470",
    "Dreamshaper.safetensors"     : "https://civitai.com/api/download/models/128713",
    "Delibrate.safetensors"     : "https://civitai.com/api/download/models/156110",
    "RevAnimated.safetensors"     : "https://civitai.com/api/download/models/46846",
    "MeinaMix.safetensors"     : "https://civitai.com/api/download/models/119057",
    "DarkshushiMix.safetensors"     : "https://civitai.com/api/download/models/93208",
    "chilloutmix_NiPrunedFp16Fix.safetensors"         : "https://huggingface.co/naonovn/chilloutmix_NiPrunedFp32Fix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors",
    "AnythingV5V3_v5PrtRE.safetensors"     : "https://huggingface.co/ckpt/anything-v5.0/resolve/main/AnythingV5V3_v5PrtRE.safetensors",
    "Anything_inkV5.safetensors"     : "https://civitai.com/api/download/models/90854",
    "ProtoGen_X5.safetensors"     :"https://huggingface.co/darkstorm2150/Protogen_x5.8_Official_Release/resolve/main/ProtoGen_X5.8-pruned-fp16.safetensors",
    "none"         : "",
}

sd_vae_url = {
    "WD-v2.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/WD-v2.vae.pt',
    "blessed-fix.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/blessed-fix.vae.pt',
    "blessed.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/blessed.vae.pt',
    "blessed2.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/blessed2.vae.pt',
    "clearvae_main.safetensors": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/clearvae_main.safetensors',
    "cute_vae.safetensors": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/cute%20vae.safetensors',
    "grapefruitVAE_v1.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/grapefruitVAE_v1.pt',
    "kl-f8-anime.ckpt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/kl-f8-anime.ckpt',
    "kl-f8-anime2.ckpt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/kl-f8-anime2.ckpt',
    "nai.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/nai.vae.pt',
    "orangemix.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/orangemix.vae.pt',
    "pastel-waifu-diffusion.vae.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/pastel-waifu-diffusion.vae.pt',
    "rmada-cold-vae.ckpt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/rmada-cold-vae.ckpt',
    "vae-ft-mse-840000-ema-pruned.ckpt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt',
    "vaeextremecolors_v10.pt": 'https://huggingface.co/Kefasu/sd-vae-collection/resolve/main/vaeextremecolors_v10.pt'
}

upscalers_urls = {
    "4x-UltraSharp.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x-UltraSharp.pth',
    "4x_CountryRoads_377000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_CountryRoads_377000_G.pth',
    "4x_Fatality_Comix_260000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_Fatality_Comix_260000_G.pth',
    "4x_RealisticRescaler_100000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_RealisticRescaler_100000_G.pth',
    "4x_Valar_v1.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_Valar_v1.pth',
    "4x_fatal_Anime_500000_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_fatal_Anime_500000_G.pth',
    "4x_foolhardy_Remacri.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_foolhardy_Remacri.pth',
    "A_ESRGAN_Single.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/A_ESRGAN_Single.pth',
    "LADDIER1_282500_G.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/LADDIER1_282500_G.pth',
    "WaifuGAN_v3_30000.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/WaifuGAN_v3_30000.pth',
    "lollypop.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/lollypop.pth',
    "sudo_rife4_269.662_testV1_scale1.pth": 'https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/sudo_rife4_269.662_testV1_scale1.pth',
}

#Extensions
!git clone https://github.com/BlafKing/sd-civitai-browser-plus /content/A666/extensions/civitai-browse
!git clone https://github.com/joebayiden/roop-wibuu /content/A666/extensions/roop
!git clone https://github.com/joebayiden/mages-browser /content/A666/extensions/image-browser
!git clone https://github.com/joebayiden/inpaint-anything /content/A666/extensions/inpaint-anything
!git clone https://github.com/joebayiden/ultimate-upscale /content/A666/extensions/ultimate-upscale
!git clone https://github.com/joebayiden/batchlink /content/A666/extensions/batchlinks
!git clone https://github.com/Bing-su/adetailer.git /content/A666/extensions/ADetailer
!git clone https://github.com/joebayiden/tag-complete /content/A666/extensions/tag-complete
!git clone https://github.com/joebayiden/controlnet  /content/A666/extensions/ControlNet
#!git clone https://github.com/hnmr293/posex /content/A666/extensions/posex
!git clone https://github.com/joebayiden/additional-networks /content/A666/extensions/addtional-networks
!git clone https://github.com/joebayiden/3d-open-pose-editor /content/A666/extensions/3d-open-pose

def wget_download():
  start_time = time.time()
  !apt install libunwind8-dev -yqq
  os.environ["LD_PRELOAD"] = "libtcmalloc.so.4"
  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  !sudo apt-get install sox ffmpeg libcairo2 libcairo2-dev
  end_time = time.time()
#  clear_output()
  print("Time spent on completing downloads using wget:", end_time-start_time,"second")

def pip_download():
  start_time = time.time()
  !pip install xformers xformers==0.0.20
  !pip install opencv-python==4.7.0.72 onnxruntime==1.15.0 onnx==1.14.0 insightface==0.7.3
  !pip install ifnude
  !pip install cython
  !pip install urllib3
  !pip install chardet
  end_time = time.time()
#  clear_output()
  print("Time spent on completing downloads using pip", end_time-start_time,"second")

def aria2_download():
  start_time = time.time()
  !aria2c --console-log-level=warn -c -x 16 -s 16 -k 1M {sd_vae_url[sd_vae]} -d /content/A666/models/VAE -o {sd_vae}
  !aria2c --console-log-level=warn -c -x 16 -s 16 -k 1M {upscalers_urls[upscalers]} -d /content/A666/models/ESRGAN -o {upscalers}
  !aria2c --console-log-level=warn -c -x 16 -s 16 -k 1M {sd_url[sd_model]}  -d /content/A666/models/Stable-diffusion -o {sd_model}
  !aria2c --console-log-level=warn -c -x 16 -s 16 -k 1M https://huggingface.co/facefusion/models/resolve/main/inswapper_128.onnx -d /content/A666/models/roop -o inswapper_128.onnx
  %cd /content/A666/extensions/ControlNet/models
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -o control_v11e_sd15_ip2p_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -o control_v11e_sd15_shuffle_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -o control_v11p_sd15_canny_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -o control_v11f1p_sd15_depth_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -o control_v11p_sd15_inpaint_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -o control_v11p_sd15_lineart_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -o control_v11p_sd15_mlsd_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -o control_v11p_sd15_normalbae_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -o control_v11p_sd15_openpose_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -o control_v11p_sd15_scribble_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -o control_v11p_sd15_seg_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -o control_v11p_sd15_softedge_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -o control_v11f1e_sd15_tile_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml -o control_v11e_sd15_ip2p_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml -o control_v11e_sd15_shuffle_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml -o control_v11p_sd15_canny_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml -o control_v11f1p_sd15_depth_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml -o control_v11p_sd15_inpaint_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml -o control_v11p_sd15_lineart_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml -o control_v11p_sd15_mlsd_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml -o control_v11p_sd15_normalbae_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml -o control_v11p_sd15_openpose_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml -o control_v11p_sd15_scribble_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml -o control_v11p_sd15_seg_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml -o control_v11p_sd15_softedge_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml -o control_v11p_sd15s2_lineart_anime_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml -o control_v11f1e_sd15_tile_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth -o t2iadapter_style_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth -o t2iadapter_sketch_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth -o t2iadapter_seg_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth -o t2iadapter_openpose_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth -o t2iadapter_keypose_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd14v1.pth -o t2iadapter_depth_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth -o t2iadapter_color_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd14v1.pth -o t2iadapter_canny_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth -o t2iadapter_canny_sd15v2.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth -o t2iadapter_depth_sd15v2.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth -o t2iadapter_sketch_sd15v2.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth -o t2iadapter_zoedepth_sd15v1.pth
  end_time = time.time()
#  clear_output()
  print("Time spent on completing downloads using aria2", end_time-start_time,"second")

def curl_download():
  start_time = time.time()
  !curl -Lo "/content/A666/models/adetailer/person_yolov8s-seg.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8s-seg.pt
  !curl -Lo "/content/A666/models/adetailer/person_yolov8n-seg.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8n-seg.pt
  !curl -Lo "/content/A666/models/adetailer/person_yolov8m-seg.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8m-seg.pt
  !curl -Lo "/content/A666/models/adetailer/hand_yolov8s.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt
  !curl -Lo "/content/A666/models/adetailer/hand_yolov8n.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8n.pt
  !curl -Lo "/content/A666/models/adetailer/face_yolov8s.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8s.pt
  !curl -Lo "/content/A666/models/adetailer/face_yolov8n_v2.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8n_v2.pt
  !curl -Lo "/content/A666/models/adetailer/face_yolov8n.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8n.pt
  !curl -Lo "/content/A666/models/adetailer/face_yolov8m.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8m.pt
  !curl -Lo "/content/A666/models/adetailer/deepfashion2_yolov8s-seg.pt" https://huggingface.co/Bingsu/adetailer/resolve/main/deepfashion2_yolov8s-seg.pt
  end_time = time.time()
#  clear_output()
  print("Time spent on completing downloads using curl", end_time-start_time,"second")

def gdrive_load():
  if os.path.exists("/content/drive/MyDrive/A666/extensions"):
    !rsync -a /content/drive/MyDrive/A666/extensions/* /content/A666/extensions
  elif not os.path.isdir("/content/drive/MyDrive/A666/extensions"):
    os.makedirs("/content/drive/MyDrive/A666/extensions")
    !rsync -a /content/drive/MyDrive/A666/extensions/* /content/A666/extensions
    print('Plugins loaded from cloud drive')
  if os.path.exists("/content/drive/MyDrive/A666/VAE"):
    !rsync -a /content/drive/MyDrive/A666/VAE/* /content/A666/models/VAE
    print('VAE loaded from cloud drive')
  elif not os.path.isdir("/content/drive/MyDrive/A666/VAE"):
    os.makedirs("/content/drive/MyDrive/A666/VAE")
    !rsync -a /content/drive/MyDrive/A666/VAE/* /content/A666/models/VAE
    print('VAE loaded from cloud drive')
  if os.path.exists("/content/drive/MyDrive/A666/embeddings"):
    !rsync -a /content/drive/MyDrive/A666/embeddings/* /content/A666/embeddings
    print('Embeddings loaded from cloud drive')
  elif not os.path.isdir("/content/drive/MyDrive/A666/embeddings"):
    os.makedirs("/content/drive/MyDrive/A666/embeddings")
    !rsync -a /content/drive/MyDrive/A666/embeddings/* /content/A666/embeddings
    print('Embeddings loaded from cloud drive')
  if os.path.exists("/content/drive/MyDrive/A666/lora"):
    !rsync -a /content/drive/MyDrive/A666/lora/* /content/A666/models/Lora
    print('lora loaded from cloud drive')
  elif not os.path.isdir("/content/drive/MyDrive/A666/lora"):
    os.makedirs("/content/drive/MyDrive/A666/lora")
    !rsync -a /content/drive/MyDrive/A666/lora/* /content/A666/models/Lora
    print('lora loaded from cloud drive')
  if os.path.exists("/content/drive/MyDrive/A666/checkpoint"):
    !rsync -a /content/drive/MyDrive/A666/checkpoint/* /content/A666/models/Stable-diffusion
    print('checkpoint loaded from cloud drive')
  elif not os.path.isdir("/content/drive/MyDrive/A666/checkpoint"):
    os.makedirs("/content/drive/MyDrive/A666/checkpoint")
    !rsync -a /content/drive/MyDrive/A666/checkpoint/* /content/A666/models/Stable-diffusion
    print('checkpoint loaded from cloud drive')

gdrive_load()

executor = concurrent.futures.ThreadPoolExecutor(max_workers=4)
task1 = executor.submit(pip_download)
task2 = executor.submit(wget_download)
task3 = executor.submit(aria2_download)
task4 = executor.submit(curl_download)
concurrent.futures.wait([task1,task2,task3,task4])

---

---

In [ ]:
#@title # [🔻] Optional: Models Downloader 🗂️
from IPython.display import clear_output
import os
import requests
from urllib.parse import unquote

def download_model(Type, DirectLink_URL):
    if Type == "None" or not DirectLink_URL:
        print("Skipping download as no valid Type or DirectLink_URL provided.")
        return

    output_path = "/content/A666/models/"

    if Type == "Checkpoint":
        output_path += "Stable-diffusion/"

    elif Type == "VAR":
        output_path += "VAE/"
    elif Type == "LoRA":
        output_path += "Lora/"
    elif Type == "LoCon/LyCORIS":
        output_path += "LyCORIS/"
    elif Type == "Textual Inversion":
        output_path += "embeddings/"
    else:
        print("Invalid type specified.")
        return

    # Create the directories if they don't exist
    os.makedirs(output_path, exist_ok=True)

    urls = [url.strip() for url in DirectLink_URL.split(",")]

    for url in urls:
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            content_disposition = response.headers.get('content-disposition')
            if content_disposition:
                filename = unquote(content_disposition.split('filename=')[1])
            else:
                filename = unquote(url.split("/")[-1])  # Extracting the filename from the URL

            # Remove double quotes and semicolons from the filename
            filename = filename.replace('"', '').replace(';', '')

            filename = os.path.join(output_path, filename)  # Modify the filename to include the output path

            if not os.path.exists(filename):
                print("Downloading file:", filename)
                chunk_size = 5242880  # 5 MB
                with open(filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=chunk_size):
                        if chunk:
                            f.write(chunk)
                print("File downloaded successfully.")
            else:
                print("File already exists:", filename)
        else:
            print("Failed to download the file:", url)

    # Clear the output to keep the notebook clean
    clear_output()

    # Print the success message
    print("✅ Done", "success")

# Call the function with the first set of parameters
Type1 = "Checkpoint"  # @param ["None", "Checkpoint", "VAE", "LoRA", "LoCon/LyCORIS", "Textual Inversion"]
DirectLink_URL1 = ""  # @param {'type': 'string'}
download_model(Type1, DirectLink_URL1)

# Call the function with the second set of parameters
Type2 = "LoRA"  # @param ["None", "Checkpoint", "VAE", "LoRA", "LoCon/LyCORIS", "Textual Inversion"]
DirectLink_URL2 = ""  # @param {'type': 'string'}
download_model(Type2, DirectLink_URL2)

import gdown

drive_url =""  # @param {'type': 'string'}
output_path = '/content/drive/MyDrive/A666/lora/zoe.safetensors'
gdown.download(drive_url, output_path, quiet=False,fuzzy=True)

---

In [ ]:
#@title # [🔻] Start Service

%cd /content/A666
use_ngrok = False  # @param {type:'boolean'}

stg = "--share --listen --xformers --no-half-vae --theme dark"
ngrok_token=""  #@param {type:"string"}

config_pathr="/content/A666/config.json"
!sed -i 's@"outdir_txt2img_samples": "outputs/txt2img-images"@"outdir_txt2img_samples": "/content/gdrive/MyDrive/A666/outputs/txt2img-images"@' {config_pathr}
!sed -i 's@"outdir_img2img_samples": "outputs/img2img-images"@"outdir_img2img_samples": "/content/gdrive/MyDrive/A666/outputs/img2img-images"@' {config_pathr}
!sed -i 's@"outdir_extras_samples": "outputs/extras-images"@"outdir_extras_samples": "/content/gdrive/MyDrive/A666/outputs/extras-images"@' {config_pathr}
!sed -i 's@"outdir_txt2img_grids": "outputs/txt2img-grids"@"outdir_txt2img_grids": "/content/gdrive/MyDrive/A666/outputs/txt2img-grids"@' {config_pathr}
!sed -i 's@"outdir_img2img_grids": "outputs/img2img-grids"@"outdir_img2img_grids": "/content/gdrive/MyDrive/A666/outputs/img2img-grids"@' {config_pathr}
!sed -i 's@"outdir_save": "log/images"@"outdir_save": "/content/gdrive/MyDrive/A666/outputs/log/images"@' {config_pathr}
!sed -i -e '''/from modules import launch_utils/a\import os''' /content/A666/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\]/\"sd_model_checkpoint\"\,\"sd_vae\"\,\"CLIP_stop_at_last_layers\"\]/g' /content/A666/modules/shared.py

if use_ngrok:
   !python launch.py {stg} --ngrok={ngrok_token}
else:
   !python launch.py {stg}